In [5]:
%matplotlib inline 
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
#from matplotlib import rcParams
#rcParams['figure.figsize'] = 30,9
#!pip install seaborn
import seaborn as sns
import csv

In [6]:
tf_out = False   # Multi regression output for tensorflow
SEA_ele = True    # True means Location to GFEM element information is extracted from SE/SEA txt files.
ele_sel = ['ELEMENTS_FRA', 'ELEMENTS_PAN1', 'ELEMENTS_PAN2'] # elements selected for output
vali_modes =  ['auto','blind_panel','blind_LC']
vali_mode = vali_modes[0]
shells = ['US','LS','SS']
shell = shells[2]
stress = False      # False if RF
name_in = "A359_MSN216_S18LHS_SS"
folder_in = '01_Input/'
folder_out = '02_Data/'
folder_in = folder_in + name_in
folder_out = folder_out + name_in

#input_name = 'A359_MSN216_S18_'
GFEM_name = 'GFEM_MSN216_' + shell
geo_name = 'geo_US_MSN321.csv'


## Select the Failure Mode to be considered

In [7]:
if tf_out is True:
    FMs = ['RF_UL_FF_MS_comp', 'RF_UL_W_MS', 'RF_UL_FF_LS', 'RF_UL_W_BUCK', 'RF_UL_FF_LB']
    print(FMs)
elif stress is False:
    FMs = ['RF_UL_FF_MS_comp', 'RF_UL_W_MS', 'RF_UL_FF_LS', 'RF_UL_JOINT_FR_to_CLIP_CL', 'RF_UL_JOINT_FR_to_CLIP_FR',
        'RF_UL_W_BUCK', 'RF_UL_FF_LB', 'RF_UL_JOINT_SK_to_CLIP_CL_R', 'RF_UL_JOINT_SK_to_CLIP_SK_R']
else:
    FMs = ['sig_FF_UL', 'sig_OF_UL']
considered_FM = FMs[0]
considered_FM

'RF_UL_FF_MS_comp'

# Data Preparation

#### ISAMI Results Data Frame

-> created by 'read_CFA' script

Set to True if the frame structure assembly should be considered upside down. In this case, the load extraction is done on the upper side of the frame, at the level of the upper stringers.
Set to False if the frame structure assembly is performed normally. In this case, the load extraction is done on the lower side of the frame, at the level of the lower stringers. See Figure 64.

Set to NodeA : the load extraction is done on the frame SEA, NodeB : the load extraction is done on the upper side of the frame (opposite node), or NodeA_and_B : double extraction at both node.

In [8]:
input_file = 'df_' + name_in  + '_cfa.pkl'
df_isami = pd.read_pickle(folder_out +'/'+input_file)
#df_isami['LC_number'] = df_isami['LC_number'].astype(int).astype(str)
# Un comment the following line for 'centered' Failure Modes, like Clip to skin joint. 
#Otherwise you will have 2 entries with alomst the same information,resulting in overfitting.
#df_isami = df_isami.loc[df_isami['Node'] == 'F']
print(df_isami.shape)
print('Number of unique LCs: '+str(len(df_isami['GFEM_Label'].unique())))
print('Number of unique locations: '+str(len(df_isami['Location'].unique())))
print('Number of unique SEA: '+str(len(df_isami['SEA'].unique())))
df_isami.columns

(7888, 41)
Number of unique LCs: 464
Number of unique locations: 17
Number of unique SEA: 17


Index(['Load_Case_ID', 'AnalysisID', 'N_Fr_UL', 'M_Fr_UL', 'T_UL',
       'f_x_Skin_L_UL', 'f_z_Skin_L_UL', 'f_xz_Skin_L_UL', 'f_x_Skin_R_UL',
       'f_z_Skin_R_UL', 'f_xz_Skin_R_UL', 'Delta_p_UL', 'T_GFEM',
       'Delta_Fx_UL', 'A_Fr', 'I_Fr', 'EA_Fr', 'yCG_Fr', 'b_eff_111_UL',
       'b_eff_121_UL', 'path', 'RF_UL_FF_MS_comp', 'RF_UL_W_MS', 'RF_UL_FF_LS',
       'RF_UL_JOINT_FR_to_CLIP_CL', 'RF_UL_JOINT_FR_to_CLIP_FR',
       'RF_UL_W_BUCK', 'RF_UL_FF_LB', 'RF_UL_JOINT_SK_to_CLIP_CL_R',
       'RF_UL_JOINT_SK_to_CLIP_SK_R', 'sig_FF_UL', 'sig_OF_UL', 'Node',
       'Location_C', 'Location_P', 'SEA', 'GFEM_Label', 'Location_C2',
       'Location_P2', 'Location_side', 'Location'],
      dtype='object')

#### RF Data Frame

In [9]:
if tf_out is False:
    df_rf = df_isami[['GFEM_Label', considered_FM, 'Location', 'SEA', 'Node']]
    print('Before cleaning:')
    print(df_rf.shape)
    df_rf = df_rf.loc[df_rf[considered_FM] != -9999.99].dropna()
else:
    df_rf = df_isami[['GFEM_Label', 'Location', 'SEA', 'Node']+FMs]
    df_rf = df_rf.loc[df_rf[considered_FM] != -9999.99].dropna()

print('After cleaning:')
print(df_rf.shape)
df_rf.head()

Before cleaning:
(7888, 5)
After cleaning:
(2154, 5)


,GFEM_Label,RF_UL_FF_MS_comp,Location,SEA,Node
6,XFB641,1795.60000,86_28_LHS,FRA-FR086-ST028-029,A
8,XFB641N,8.33629,86_28_LHS,FRA-FR086-ST028-029,A
11,XFB642N,12.81460,86_28_LHS,FRA-FR086-ST028-029,A
12,GKN641,31.28570,86_28_LHS,FRA-FR086-ST028-029,A
20,GKN643,84.98980,86_28_LHS,FRA-FR086-ST028-029,A


#### LCDEF Data Frame
Note: This is a work around. In future you should read this information directly from the LCDEF file.
This is required for new op2/new load cases.

In [10]:
df_lcdef = pd.read_csv('04_LCDEF/LCDEF_event.csv')
df_lcdef = df_lcdef[['GFEM_Label','Event', 'LC_type', 'Thermal', 'Pressure', 'LC_number']].dropna()
lb_make = LabelEncoder()
df_lcdef['Thermal_code'] = lb_make.fit_transform(df_lcdef['Thermal'])
df_lcdef['Pressure_code'] = lb_make.fit_transform(df_lcdef['Pressure'])
df_lcdef['LC_number'] = df_lcdef['LC_number'].astype(int).astype(str)
#df_lcdef = df_lcdef[['LC_number', 'Thermal_code', 'Pressure_code', 'LC_type']]
print(df_lcdef.shape)
df_lcdef.head()

(464, 8)


,GFEM_Label,Event,LC_type,Thermal,Pressure,LC_number,Thermal_code,Pressure_code
0,ZCT,Internal Pressure,DP,No-Thermal,Pressurized,501,1,1
1,DT1.15,Internal Pressure,DP,No-Thermal,Pressurized,502,1,1
2,FATIGUE,Fatigue,FAT,FAT,Pressurized,503,0,1
3,FATGL1,Fatigue,FAT,FAT,Pressurized,504,0,1
4,FATGL2,Fatigue,FAT,FAT,Pressurized,505,0,1


## Elements Data Frame (SEA + SE)

In [22]:
if SEA_ele is False:
    input_file = name_in + '_Ele.csv'
    df_isami_elements1 = pd.read_csv(folder_out+ '/QVLV/'+ input_file, index_col=0)
    df_isami_elements = df_isami_elements1[['Location','FrameEl', 'SkinEl_LHS', 'SkinEl_RHS']].copy()
    df_isami_elements[['FrameEl', 'SkinEl_LHS', 'SkinEl_RHS']] = df_isami_elements[['FrameEl', 'SkinEl_LHS', 'SkinEl_RHS']].astype(int).astype(str)
    df_isami_elements.rename(columns={'FrameEl': 'ELEMENTS_FRA_1','SkinEl_LHS':'ELEMENTS_PAN1_1', 'SkinEl_RHS':'ELEMENTS_PAN2_1' }, inplace=True)
    print(df_isami_elements.shape)
    df_isami_elements.tail()

In [24]:
parent_dir = folder_in + '/SEA_SE_FE'
subject_dirs = [x[0] for x in sorted(os.walk(parent_dir))]
i = 0
j = 0
filelist = []
for dir in subject_dirs:
    csv_files = [os.path.join(dir, txt) for txt in os.listdir(dir) if os.path.isfile(os.path.join(dir, txt)) 
                 and txt.endswith('.txt') and ('out' not in txt)
                ]
    csv_files = sorted(csv_files)
    for file in csv_files:
        with open(file, "rt") as fin:
            file_out = file[:-4]+'out.txt'
            with open(file_out, "wt") as fout:
                for line in fin:
                    #line = line.replace('#SE', 'SE')
                    line = line.replace('$', '#')
                    fout.write(line)
        f_name = file.split('/')
        if any(x in f_name[-1] for x in ['FE2SE','SE2FE']):
            df_FE2SE = pd.read_csv(file_out,sep=":",header=None, skip_blank_lines=True, comment="#")
            df_FE2SE['FILE'] = f_name[-1]
            if i == 0:
                df1 = df_FE2SE
            else:
                df1 = df1.append(df_FE2SE)
            i = i + 1
        elif any(x in f_name[-1] for x in ['SEA2SE','SE2SEA']):
            df_SEA2SE = pd.read_csv(file_out,sep=":",header=None, skip_blank_lines=True, comment="#")
            df_SEA2SE['FILE'] = f_name[-1]
            if j == 0:
                df2 = df_SEA2SE
            else:
                df2 = df2.append(df_SEA2SE)
            j = j + 1
                
SEA_list = ['PAN1','PAN2','FRA','STR1', 'STR2', 'STR3', 'STR4']
df1[['TYPE','SE']] = df1[0].str.split(" ",1, expand = True).astype(str)
df2[['TYPE','SEA']] = df2[0].str.split(" ",1, expand = True).astype(str)
df1['SE'] = df1['SE'].str.replace(' ','')
df2['SEA'] = df2['SEA'].str.replace(' ','')
df1[2] = df1[1].str.replace(' ','').str.split(",", expand = False)
df2[SEA_list]  = df2[1].str.replace(' ','').str.split(",", expand = True).astype(str)
df1.rename(columns={2: 'ELEMENTS'}, inplace=True)
df2 = df2[(df2.TYPE == 'AirbusSEA_Frame')]

In [25]:
print(any(df1['SE'].duplicated())) 
ids = df1['SE']
df1[ids.isin(ids[ids.duplicated()])]
#only keep the first(~)/last entry:
df1 = df1[~df1['SE'].duplicated()]

True


In [26]:
print(any(df2['SEA'].duplicated())) 
#df2_d = df2[df2['SEA'].duplicated()]
ids = df2['SEA']
df2[ids.isin(ids[ids.duplicated()])]
#only keep the first(~)/last entry:
df2 = df2[~df2['SEA'].duplicated()]

True


In [27]:
i = 0
for item in SEA_list:
    if i == 0:
        df_ele = pd.merge(df2[['TYPE', 'SEA']+SEA_list], df1[['SE', 'ELEMENTS']],
                  how='left', left_on=item, right_on='SE'
                 ).drop(['SE'], axis=1)
        
    else:
        df_ele = pd.merge(df_ele, df1[['SE', 'ELEMENTS']],
          how='left', left_on=item, right_on='SE'
         ).drop(['SE'], axis=1)
    df_ele.rename(columns={'ELEMENTS': 'ELEMENTS_'+item}, inplace=True)
    i = i +1
print(df_ele.shape)
df_ele.head()

(713, 16)


,TYPE,SEA,PAN1,PAN2,FRA,STR1,STR2,STR3,STR4,ELEMENTS_PAN1,ELEMENTS_PAN2,ELEMENTS_FRA,ELEMENTS_STR1,ELEMENTS_STR2,ELEMENTS_STR3,ELEMENTS_STR4
0,AirbusSEA_Frame,FRA-FR072-ST006-007,Pan-FR071-072-ST006-007,Pan-FR072-073-ST006-007,Fra-FR072-ST006-007,Str-FR071-072-ST007,Str-FR071-072-ST006,Str-FR072-073-ST007,Str-FR072-073-ST006,[7700730],[7800730],[7800710],[7700820],[7700720],[7800820],[7800720]
1,AirbusSEA_Frame,FRA-FR073-ST006-007,Pan-FR072-073-ST006-007,Pan-FR073-074-ST006-007,Fra-FR073-ST006-007,Str-FR072-073-ST007,Str-FR072-073-ST006,Str-FR073-074-ST007,Str-FR073-074-ST006,[7800730],[7900730],[7900710],[7800820],[7800720],[7900820],[7900720]
2,AirbusSEA_Frame,FRA-FR074-ST006-007,Pan-FR073-074-ST006-007,Pan-FR074-075-ST006-007,Fra-FR074-ST006-007,Str-FR073-074-ST007,Str-FR073-074-ST006,Str-FR074-075-ST007,Str-FR074-075-ST006,[7900730],[8000730],[8000710],[7900820],[7900720],[8000820],[8000720]
3,AirbusSEA_Frame,FRA-FR075-ST006-007,Pan-FR074-075-ST006-007,Pan-FR075-076-ST006-007,Fra-FR075-ST006-007,Str-FR074-075-ST007,Str-FR074-075-ST006,Str-FR075-076-ST007,Str-FR075-076-ST006,[8000730],[8100730],[8100710],[8000820],[8000720],[8100820],[8100720]
4,AirbusSEA_Frame,FRA-FR076-ST006-007,Pan-FR075-076-ST006-007,Pan-FR076-077-ST006-007,Fra-FR076-ST006-007,Str-FR075-076-ST007,Str-FR075-076-ST006,Str-FR076-077-ST007,Str-FR076-077-ST006,[8100730],[8200730],[8200710],[8100820],[8100720],[8200820],[8200720]


## Geometry Data Frame

-> basis: ISAMI Results Data Frame -> to be merged with GFEM Data Frame in order to get geometry features

In [28]:
geom_par = ['A_Fr','I_Fr', 'EA_Fr', 'yCG_Fr', 'b_eff_111_UL', 'b_eff_121_UL']
list1 = ['GFEM_Label','Location', 'SEA']
df_geom = df_isami[geom_par + list1]
df_geom = df_geom.loc[df_geom['GFEM_Label'] == 'DT1.15'].dropna().drop_duplicates().reset_index(drop=True)
print(df_geom.shape)
# df_geom.to_csv('df_geom.csv')
a1 = df_geom['Location'].unique()
print(len(a1))
#pd.DataFrame(a1).to_clipboard()
df_geom.head()

(17, 9)
17


,A_Fr,I_Fr,EA_Fr,yCG_Fr,b_eff_111_UL,b_eff_121_UL,GFEM_Label,Location,SEA
0,812.486,1199760.0,70540700.0,107.9150,317.5,317.5,DT1.15,86_28_LHS,FRA-FR086-ST028-029
1,235.392,196841.0,19781600.0,97.4703,317.5,317.5,DT1.15,72_15_LHS,FRA-FR072-ST015-016
2,224.600,187589.0,18714200.0,95.8606,317.5,317.5,DT1.15,73_15_LHS,FRA-FR073-ST015-016
3,218.417,181507.0,17539800.0,93.9115,317.5,317.5,DT1.15,74_15_LHS,FRA-FR074-ST015-016
4,168.766,137862.0,9719730.0,91.3548,317.5,317.5,DT1.15,75_15_LHS,FRA-FR075-ST015-016


### Geometry from fishtail file

In [29]:
df_ft = pd.read_pickle(folder_out+'/' +'df_'+name_in + '_geo.pkl')
df_ft.head()

,AIRBUSSEA_FRAME,CLIPPROFILE,CLIPLENGTH (mm),CLIPPOSITIONX (mm),JOINT_FRA_CLIP,FRA_CLIP_HEADSIDE,JOINT_FRACLIP_SKIN,FRACLIP_SKIN_HEADSIDE,LOWERSTAB,LOWERSTABPOSITIONX (mm),...,MATERIAL TYPE,FAMILY,BA (mm),BF (mm),H (mm),-45.0,0.0,45.0,90.0,SEA_MSN216
0,FRA-Fra-FR072-ST008-009,Clip_C72_P8,208.67,0,FA_C72P_8,Frame,FP_C72P_8,Skin,Dummy Stabilo,208.67,...,Compo,Z,14.906,29.118,80.0,2.0,4.0,2.0,2.0,FRA-FR072-ST008-009
1,FRA-Fra-FR072-ST009-010,Clip_C72_P9,220.00,0,FA_C72P_9,Frame,FP_C72P_9,Skin,Dummy Stabilo,220.00,...,Compo,Z,14.906,28.934,80.0,2.0,4.0,2.0,2.0,FRA-FR072-ST009-010
2,FRA-Fra-FR072-ST010-011,Clip_C72_P10,208.67,0,FA_C72P_10,Frame,FP_C72P_10,Skin,Lower_Stabilo_C72P10,208.67,...,Compo,Z,14.906,31.934,80.0,2.0,4.0,2.0,2.0,FRA-FR072-ST010-011
3,FRA-Fra-FR072-ST011-012,Clip_C72_P11,208.67,0,FA_C72P_11,Frame,FP_C72P_11,Skin,Lower_Stabilo_C72P11,208.67,...,Compo,Z,14.906,31.934,80.0,2.0,4.0,2.0,2.0,FRA-FR072-ST011-012
4,FRA-Fra-FR072-ST012-013,Clip_C72_P12,208.67,0,FA_C72P_12,Frame,FP_C72P_12,Skin,Lower_Stabilo_C72P12,208.67,...,Compo,Z,14.906,29.934,80.0,2.0,4.0,2.0,2.0,FRA-FR072-ST012-013


In [30]:
df_geoc = pd.merge(df_ft, df_geom, 
                        how='left', left_on=['SEA_MSN216'], right_on=['SEA']).drop('SEA_MSN216', axis=1)
df_geoc = df_geoc.reset_index(drop=True)
print(df_geoc.shape)
df_geoc.head()

(575, 43)


,AIRBUSSEA_FRAME,CLIPPROFILE,CLIPLENGTH (mm),CLIPPOSITIONX (mm),JOINT_FRA_CLIP,FRA_CLIP_HEADSIDE,JOINT_FRACLIP_SKIN,FRACLIP_SKIN_HEADSIDE,LOWERSTAB,LOWERSTABPOSITIONX (mm),...,90.0,A_Fr,I_Fr,EA_Fr,yCG_Fr,b_eff_111_UL,b_eff_121_UL,GFEM_Label,Location,SEA
0,FRA-Fra-FR072-ST008-009,Clip_C72_P8,208.67,0,FA_C72P_8,Frame,FP_C72P_8,Skin,Dummy Stabilo,208.67,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FRA-Fra-FR072-ST009-010,Clip_C72_P9,220.00,0,FA_C72P_9,Frame,FP_C72P_9,Skin,Dummy Stabilo,220.00,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FRA-Fra-FR072-ST010-011,Clip_C72_P10,208.67,0,FA_C72P_10,Frame,FP_C72P_10,Skin,Lower_Stabilo_C72P10,208.67,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FRA-Fra-FR072-ST011-012,Clip_C72_P11,208.67,0,FA_C72P_11,Frame,FP_C72P_11,Skin,Lower_Stabilo_C72P11,208.67,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FRA-Fra-FR072-ST012-013,Clip_C72_P12,208.67,0,FA_C72P_12,Frame,FP_C72P_12,Skin,Lower_Stabilo_C72P12,208.67,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### OLD: Geometry from fishtail file

In [31]:
'''df_ft = pd.read_csv(folder_out+'/'+geo_name)
params = df_ft['param'].unique().tolist()
para = params[0]

# This code works for A351 clip geo
#df_ft['Location_C'] = df_ft['Profile'].str.split('_').str[-2].str.replace('.','0').str.replace('C','')
df_ft.loc[df_ft['Profile'].str.contains(r'Clp_'),'Location_C'] = df_ft['Profile'].str.split('_').str[-2].str.replace('.','0').str.replace('C','') 
df_ft.loc[df_ft['Profile'].str.contains(r'IF_FC_LJ'),'Location_C'] = df_ft['Profile'].str.split('_').str[-3].str.replace('.','0').str.replace('C','')
#df_ft['Location_P'] = df_ft['Profile'].str.split('_').str[-1].str.replace(r'Pr*','')
df_ft.loc[df_ft['Profile'].str.contains(r'Clp_'),'Location_P'] = df_ft['Profile'].str.split('_').str[-1].str.replace(r'Pr*','')
df_ft.loc[df_ft['Profile'].str.contains(r'IF_FC_LJ'),'Location_P'] = df_ft['Profile'].str.split('_').str[-2].str.replace(r'Pr*','')

df_ft.loc[df_ft['Profile'].str.contains(r'_P\d'),'Location_side'] = 'LHS'
df_ft.loc[df_ft['Profile'].str.contains(r'_Pr'),'Location_side'] = 'RHS'
df_ft.loc[df_ft['Profile'].str.contains(r'_TL'),'Location_side'] = 'LHS'
df_ft['Location'] = df_ft['Location_C'] + "_" + df_ft['Location_P'] + "_" + df_ft['Location_side']
dfclp = df_ft[['Profile','Location_C','Location','MatType','Family', 'h', 'tw'] ]
dfclp.isnull().sum()

df_ft.loc[df_ft['P_pos'].str.contains(r'P\d'),'Location_side'] = 'LHS'
df_ft.loc[df_ft['P_pos'].str.contains(r'Pr'),'Location_side'] = 'RHS'
df_ft.loc[df_ft['P_pos'].str.contains(r'TL'),'Location_side'] = 'LHS'
df_ft.loc[df_ft['P_pos'].str.contains(r'rTL|TLr'),'Location_side'] = 'RHS'
df_ft['Location_P'] = df_ft['P_pos'].str.replace(r'Pr?|r','')
df_ft.loc[df_ft['Location_P'].str.contains(r'TL'),'Location_P'] = '0'
df_ft = df_ft.drop(['P_pos'], axis=1)
df_ft2 = df_ft.melt(id_vars=['Location_P','Location_side', 'param'])
df_ft2['Location'] = df_ft2['variable'] + "_" + df_ft2['Location_P'] + "_" + df_ft2['Location_side']
df_ft2 = df_ft2.pivot(index='Location',columns='param', values='value')
df_ft2.head()
'''

'df_ft = pd.read_csv(folder_out+\'/\'+geo_name)\nparams = df_ft[\'param\'].unique().tolist()\npara = params[0]\n\n# This code works for A351 clip geo\n#df_ft[\'Location_C\'] = df_ft[\'Profile\'].str.split(\'_\').str[-2].str.replace(\'.\',\'0\').str.replace(\'C\',\'\')\ndf_ft.loc[df_ft[\'Profile\'].str.contains(r\'Clp_\'),\'Location_C\'] = df_ft[\'Profile\'].str.split(\'_\').str[-2].str.replace(\'.\',\'0\').str.replace(\'C\',\'\') \ndf_ft.loc[df_ft[\'Profile\'].str.contains(r\'IF_FC_LJ\'),\'Location_C\'] = df_ft[\'Profile\'].str.split(\'_\').str[-3].str.replace(\'.\',\'0\').str.replace(\'C\',\'\')\n#df_ft[\'Location_P\'] = df_ft[\'Profile\'].str.split(\'_\').str[-1].str.replace(r\'Pr*\',\'\')\ndf_ft.loc[df_ft[\'Profile\'].str.contains(r\'Clp_\'),\'Location_P\'] = df_ft[\'Profile\'].str.split(\'_\').str[-1].str.replace(r\'Pr*\',\'\')\ndf_ft.loc[df_ft[\'Profile\'].str.contains(r\'IF_FC_LJ\'),\'Location_P\'] = df_ft[\'Profile\'].str.split(\'_\').str[-2].str.replace(r\'Pr*\',\'\')\n\ndf_ft.

## GFEM Data Frame

-> created by the 'read_GFEM' script

In [32]:
input_file = 'df_' + GFEM_name +  '_frames.csv'
df_gfem_frames = pd.read_csv(folder_out+'/'+input_file).drop(['Unnamed: 0'], axis=1)
df_gfem_frames['Loadcase'] = df_gfem_frames['Loadcase'].astype(int).astype(str)
df_gfem_frames['ElementID'] = df_gfem_frames['ElementID'].astype(int).astype(str)
df_gfem_frames = df_gfem_frames.drop(['bending_moment_a2', 'bending_moment_b2', 'shear2', 'torque'], axis=1)
df_gfem_frames.head(1)

FileNotFoundError: File b'02_Data/A359_MSN216_S18LHS_SS/df_GFEM_MSN216_SS_frames.csv' does not exist

In [33]:
input_file = 'df_' + GFEM_name + '_skin.csv'
df_gfem_skin = pd.read_csv(folder_out+'/'+input_file).drop(['Unnamed: 0'], axis=1)
df_gfem_skin['Loadcase'] = df_gfem_skin['Loadcase'].astype(int).astype(str)
df_gfem_skin['ElementID'] = df_gfem_skin['ElementID'].astype(int).astype(str)
#df_gfem_skin.head()

FileNotFoundError: File b'02_Data/A359_MSN216_S18LHS_SS/df_GFEM_MSN216_SS_skin.csv' does not exist

## Database Join

Note:  Skin_LHS -> _x      |      Skin_RHS -> _y

        

In [37]:
#features_geom = ['A_Fr', 'I_Fr', 'EA_Fr', 'yCG_Fr', 'ba','hclp','nofastfr','nofastsk','tclp','tfr','tskin']
features_geom = ['A_Fr', 'I_Fr', 'EA_Fr', 'CLIPLENGTH (mm)','OFFSETY0 (mm)', 'OFFSETZ0 (mm)', 'MATERIAL',
                'T (mm)', 'TW (mm)'
                ]
feat_loc_list =  ['SEA'] + features_geom
if SEA_ele is True: pos_var = ['Location', 'Node', 'SEA']
else: pos_var = ['Location', 'Node']

if tf_out is False: #write out all Failure Modes
    df_combined = pd.merge(df_rf[['GFEM_Label', considered_FM]+ pos_var], df_geoc[feat_loc_list],
                         how='left', left_on=['SEA'], right_on=['SEA'])
else: # write only 1 FM
    #df_combined = pd.merge(df_rf[['LC_number'] + pos_var + FMs], df_geoc[feat_loc_list],
    df_combined = pd.merge(df_rf[['GFEM_Label'] + pos_var + FMs], df_geoc[feat_loc_list],
                         how='left', left_on=['SEA'], right_on=['SEA'])

df_combined = pd.merge(df_combined, df_lcdef[['LC_number', 'Thermal_code', 'Pressure_code', 'LC_type', 'GFEM_Label']], 
                        how='left', on=['GFEM_Label'])
if SEA_ele is True:
    df_combined = pd.merge(df_combined, df_ele[['SEA'] + ele_sel], how='left', on=['SEA'])
    df_combined[['ELEMENTS_FRA_1','ELEMENTS_FRA_2']] = pd.DataFrame(df_combined['ELEMENTS_FRA'].values.tolist(), index= df_combined.index)
    df_combined[['ELEMENTS_PAN1_1', 'ELEMENTS_PAN1_2']] = pd.DataFrame(df_combined['ELEMENTS_PAN1'].values.tolist(), index= df_combined.index)
    df_combined[['ELEMENTS_PAN2_1', 'ELEMENTS_PAN2_2']] = pd.DataFrame(df_combined['ELEMENTS_PAN2'].values.tolist(), index= df_combined.index)
    df_hv = df_combined
else:
    df_combined = pd.merge(df_combined, df_isami_elements, how='left', left_on=['Location'],
                      right_on=['Location'])

ValueError: Columns must be same length as key

In [38]:
### START Load Extraction
#df_combined = pd.merge(df_combined, df_gfem_frames, how='left', left_on=['LC_number', 'FrameEl'],
#                      right_on=['Loadcase', 'ElementID'])
df_combined1 = pd.merge(df_combined.loc[df_combined['Node'] == 'F'], 
                        df_gfem_frames.loc[:, df_gfem_frames.columns != 'bending_moment_b1'], how='left', left_on=['LC_number', 'ELEMENTS_FRA_1'],
                      right_on=['Loadcase', 'ElementID'])
df_combined1.rename(columns={'bending_moment_a1': 'bending_moment'}, inplace=True)
df_combined2 = pd.merge(df_combined.loc[df_combined['Node'] == 'T'], 
                        df_gfem_frames.loc[:, df_gfem_frames.columns != 'bending_moment_a1'], how='left', left_on=['LC_number', 'ELEMENTS_FRA_1'],
                      right_on=['Loadcase', 'ElementID'])
df_combined2.rename(columns={'bending_moment_b1': 'bending_moment'}, inplace=True)
df_combined = pd.concat([df_combined1, df_combined2])
### END Load Extraction

print(df_combined.shape)
#df4 = df_combined
###   ATTENTION!!!! drop all na!
#df_combined = df_combined.dropna()
print(df_combined.shape)
df_combined = pd.merge(df_combined, df_gfem_skin, how='left', left_on=['LC_number', 'ELEMENTS_PAN1_1'],
                      right_on=['Loadcase', 'ElementID'])
df_combined = pd.merge(df_combined, df_gfem_skin, how='left', left_on=['LC_number', 'ELEMENTS_PAN2_1'],
                      right_on=['Loadcase', 'ElementID'])
print(df_combined.shape)
print(df_combined.columns)
#df4[df4.isnull().any(axis=1)]
df_combined.head()

NameError: name 'df_gfem_frames' is not defined

In [26]:
RFmax = 20
features_loads = ['bending_moment', #'bending_moment_b1', 
                  'shear1', 'axial',
                  'mx_x', 'my_x', 'mxy_x', 'mx_y', 'my_y', 'mxy_y']
features_lcdef = ['Pressure_code', 'Thermal_code']
col_pp = ['LC_number','GFEM_Label','LC_type'] + pos_var
features = features_geom + features_loads + features_lcdef
if tf_out is False:
    columns_all = [considered_FM] + features
else:
    columns_all = FMs + features
col_pp2 = columns_all + col_pp
df_all1 = df_combined[col_pp2].dropna()

if stress is False:
    if tf_out is False:
        df_all1 = df_all1.loc[df_combined[considered_FM] != -9999.99]
        df_all = df_all1.loc[df_combined[considered_FM] < RFmax]
    else:
        df_all1[FMs] = df_all1[FMs].replace(-9999.99, RFmax)
        df_all1[FMs] = df_all1[FMs].apply(lambda x: np.where(x < RFmax,x,RFmax))
        df_all = df_all1
else:
    df_all = df_all1
df_features = df_all[features]
df_feat_pp = df_all[features + col_pp]
if tf_out is False:
    df_result = df_all[considered_FM]
else:
    df_result = df_all[FMs]
print(df_all.shape)
print(df_features.shape)
print(df_result.shape)

df_all.columns

(187721, 29)
(187721, 22)
(187721,)


Index(['RF_UL_FF_MS_comp', 'A_Fr', 'I_Fr', 'EA_Fr', 'yCG_Fr', 'ba', 'hclp',
       'nofastfr', 'nofastsk', 'tclp', 'tfr', 'tskin', 'bending_moment',
       'shear1', 'axial', 'mx_x', 'my_x', 'mxy_x', 'mx_y', 'my_y', 'mxy_y',
       'Pressure_code', 'Thermal_code', 'LC_number', 'GFEM_Label', 'LC_type',
       'Location', 'Node', 'SEA'],
      dtype='object')

In [27]:
df_all.head()

,RF_UL_FF_MS_comp,A_Fr,I_Fr,EA_Fr,yCG_Fr,ba,hclp,nofastfr,nofastsk,tclp,...,my_y,mxy_y,Pressure_code,Thermal_code,LC_number,GFEM_Label,LC_type,Location,Node,SEA
2,15.18860,217.229,181793.0,12758500.0,97.1372,25.244,78.089,4.0,4.0,1.55,...,66.655418,-10.197708,0,3,512,XFB642N,XFB,72_1_LHS,F,SEA_Frame_C72_P1
3,7.76998,217.229,181793.0,12758500.0,97.1372,25.244,78.089,4.0,4.0,1.55,...,54.018757,-43.797047,0,1,513,GKN641,GKN,72_1_LHS,F,SEA_Frame_C72_P1
4,10.47610,217.229,181793.0,12758500.0,97.1372,25.244,78.089,4.0,4.0,1.55,...,55.558369,-43.498177,0,2,515,GKN641P,GKN,72_1_LHS,F,SEA_Frame_C72_P1
5,4.52645,217.229,181793.0,12758500.0,97.1372,25.244,78.089,4.0,4.0,1.55,...,38.438648,50.492756,0,1,517,GKN642,GKN,72_1_LHS,F,SEA_Frame_C72_P1
6,5.32827,217.229,181793.0,12758500.0,97.1372,25.244,78.089,4.0,4.0,1.55,...,39.978260,50.791626,0,2,519,GKN642P,GKN,72_1_LHS,F,SEA_Frame_C72_P1


In [28]:
#Collect the LC's, which results in the lowest 5% of RF for each location.
if vali_mode == "blind_LC":
    df_all_grp = df_all[[considered_FM,'Location','LC_number','LC_name']].groupby(['Location'])
    df_all_stat = df_all_grp.describe(percentiles=[.05, .5, .95])
    df_all_stat.columns = df_all_stat.columns.droplevel()
    df_blindLC = pd.DataFrame()
    for key, group in df_all_grp:
        df = df_all_grp.get_group(key)
        RF_blind = df_all_stat.loc[key,'5%']
        if RF_blind > 1.1: RF_blind = 1.1
        df = df.loc[df[considered_FM] < RF_blind]
        df_blindLC = df_blindLC.append(df)
        #r2 = r2_score(df['RF_calculated'], df['RF_predicted'])
        #df_r2.loc[key] = r2
    print(df_blindLC.shape)
    blind = df_blindLC['LC_name'].unique().tolist()
    print(len(blind))

## Train/Test Split

In [29]:
if vali_mode == "auto":
    X_train, X_test_pp, y_train, y_test = train_test_split(df_feat_pp, df_result, test_size = 0.25, random_state = 6)
    X_train, X_test, y_train, y_test = train_test_split(df_features, df_result, test_size = 0.25, random_state = 6)
    
elif vali_mode == "blind_LC":
    #blind =['XLF655P', 'GVB653']
    ######################################
    X_train = df_features.loc[~df_all['LC_name'].isin(blind),:]
    X_test = df_features.loc[df_all['LC_name'].isin(blind),:]
    X_test_pp = df_feat_pp.loc[df_all['LC_name'].isin(blind),:]
    y_train = df_result.loc[~df_all['LC_name'].isin(blind),]
    y_test = df_result.loc[df_all['LC_name'].isin(blind),]

else:
    blind =['88_2_LHS', '74_6_RHS']
    ######################################
    X_train = df_features.loc[~df_all['Location'].isin(blind),:]
    X_test = df_features.loc[df_all['Location'].isin(blind),:]
    X_test_pp = df_feat_pp.loc[df_all['Location'].isin(blind),:]
    y_train = df_result.loc[~df_all['Location'].isin(blind),]
    y_test = df_result.loc[df_all['Location'].isin(blind),]

input_name = name_in + '_'
X_train.to_pickle(folder_out + '/' + input_name + "X_train.pkl")
X_test.to_pickle(folder_out + '/' + input_name + "X_test.pkl")
X_test_pp.to_pickle(folder_out + '/' + input_name + "X_test_pp.pkl")
y_train.to_pickle(folder_out + '/' + input_name + "y_train.pkl")
y_test.to_pickle(folder_out + '/' + input_name + "y_test.pkl")
print(features)
    
print(X_train.shape)
print(X_test.shape)
print(X_test_pp.shape)
print(y_train.shape)
print(y_test.shape)

['A_Fr', 'I_Fr', 'EA_Fr', 'yCG_Fr', 'ba', 'hclp', 'nofastfr', 'nofastsk', 'tclp', 'tfr', 'tskin', 'bending_moment', 'shear1', 'axial', 'mx_x', 'my_x', 'mxy_x', 'mx_y', 'my_y', 'mxy_y', 'Pressure_code', 'Thermal_code']
(140790, 22)
(46931, 22)
(46931, 28)
(140790,)
(46931,)


## Create Hyperview results file

In [31]:
out_file = folder_out + '/' + input_name + "hvout.hwascii"
header = ['ALTAIR ASCII FILE ',
        '$TITLE = CFA Analysis Min RF MSN216',
        '$SUBCASE = 1 ' + considered_FM  ,
        '$BINDING = ELEMENT ',
        '$COLUMN_INFO=   ENTITY_ID ',
        '$RESULT_TYPE =  RF, SLoadcase'
         ]

In [33]:
df_SEA2EL = df_hv[['Location', 'SEA', 'ELEMENTS_FRA_1', 'ELEMENTS_FRA_2']].drop_duplicates(['SEA'])
df_SEA2EL.to_pickle(folder_out + '/' + input_name + "SEA2EL.pkl")

infos = ['GFEM_Label', 'Location', 'Node', 'SEA', 'LC_number', 'LC_type',
       'ELEMENTS_FRA_1', 'ELEMENTS_FRA_2']
df_hv = df_hv[infos + FMs]
df_hv[FMs] = df_hv[FMs].replace(-9999.99, RFmax)
grphv = df_hv.groupby(['Location'])
i = 0
j = 0
w_mode = 'w'
for FM in FMs:
    df2 = pd.DataFrame()
    if i == 1: 
        w_mode = 'a'
        j = 1
    header[2] = '$SUBCASE = ' + str(i+1) + ' ' + FM
    with open(out_file, w_mode) as output:
        writer = csv.writer(output, lineterminator='\n')
        for val in header[j:]:
            writer.writerow([val])
    
    for key, group in grphv:
        df1 = grphv.get_group(key)
        df2 = df2.append(df1[infos+[FM]].nsmallest(1, FM))
    df2 = df2.set_index('Location')
    df3 = df2[['ELEMENTS_FRA_1', FM, 'LC_number']]
    df3.to_csv(out_file,sep=' ', mode='a', header=False, float_format='%.2f', index=False)
    df4 = df2[['ELEMENTS_FRA_2', FM, 'LC_number']]
    df4 = df4[df4['ELEMENTS_FRA_2'].notnull()]
    df4.to_csv(out_file,sep=' ', mode='a', header=False, float_format='%.2f', index=False)
    i = i+1


KeyError: "['RF_UL_W_MS' 'RF_UL_FF_LS' 'RF_UL_JOINT_FR_to_CLIP_CL'\n 'RF_UL_JOINT_FR_to_CLIP_FR' 'RF_UL_W_BUCK' 'RF_UL_FF_LB'\n 'RF_UL_JOINT_SK_to_CLIP_CL_R' 'RF_UL_JOINT_SK_to_CLIP_SK_R'] not in index"